In [ ]:
5.
- staticmethod, classmethod
- descriptors
- custom containers
- subclasses and MRO
- metaclasses

## staticmethod, classmethod
Methods without `self`

In [68]:
import json

class MessageBox:
    def __init__(self, title_text, body_text, dismiss_button="Okay"):
        self.title = title_text
        self.body = body_text
        self.buttons = [dismiss_button]

    def __repr__(self):
        return f'<{self.__class__.__name__} {self.title}>'
    
    @classmethod
    def from_json(cls, jtext):
        if not cls.valid_json(jtext):
            raise ValueError('Invalid JSON for this class')
        return cls(**json.loads(jtext))
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button'}
        return required <= set(data) <= everything

m = MessageBox.from_json('''
{
  "title_text": "Alert",
  "body_text": "Something bad happened"
}''')
m

<MessageBox Alert>

In [69]:
m.buttons

['Okay']

In [70]:
MessageBox.valid_json('42')

False

In [64]:
MessageBox.valid_json('{"title_text":"a","body_text":"b"}')

True

In [71]:
MessageBox.from_json('{"title_text":"hello"}')

ValueError: Invalid JSON for this class

In [72]:
class ConfirmBox(MessageBox):
    def __init__(self, title_text, body_text,
                 dismiss_button='Cancel', confirm_button='Proceed'):
        super().__init__(title_text, body_text, dismiss_button)
        self.buttons.append(confirm_button)
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button', 'confirm_button'}
        return required <= set(data) <= everything

c = ConfirmBox.from_json('''
{
  "title_text": "Danger",
  "body_text": "Vent radioactive gas?",
  "confirm_button": "Yes I'm sure"
}''')
c

<ConfirmBox Danger>

In [73]:
c.buttons

['Cancel', "Yes I'm sure"]

In [75]:
ConfirmBox.valid_json(
    '{"title_text":"hi","body_text":"how are you?",'
    '"confirm_button":"good","dismiss_button":"bad"}')

True

## Descriptors
The power behind `property`, `classmethod` and `staticmethod`

In [ ]:
class Tripwire:
    def __get__(self):